In [1]:
import pandas as pd
champ_data_df = pd.read_csv("new_regression_data.csv",index_col=0)
champ_data_df

,Role of champion_role-jungle,Role of champion_role-mid,Role of champion_role-support,Role of champion_role-top,Win rate of champion,Pick rate of champion,Move. speed,Attack range,Base AS,AS ratio,Min_Health,Max_Health,Min_Health_regen,Max_Health_regen,Min_Armor,Max_Armor,Min_Magic_resist,Max_Magic_resist,Min_Attack_damage,Max_Attack_damage
0,0,0,1,0,0.534,0.153,330,550,0.625,1.000,525.0,1783.0,6.0,16.20,29.0,91.9,30.0,38.50,47.0,91.2
1,0,0,1,0,0.523,0.198,335,450,0.625,1.000,561.0,2142.0,7.0,16.35,28.0,28.0,30.0,38.50,56.0,93.4
2,0,0,0,1,0.543,0.078,340,175,0.625,1.000,620.0,2048.0,8.0,16.50,36.0,87.0,32.0,44.75,66.0,142.5
3,1,0,0,0,0.511,0.156,345,175,0.670,1.000,580.0,2110.0,7.0,19.75,36.0,104.0,32.0,53.25,66.0,120.4
4,0,1,0,0,0.489,0.111,345,175,0.625,1.000,550.0,1995.0,7.5,20.25,28.0,85.8,32.0,53.25,60.0,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,0,1,0,0,0.519,0.007,335,550,0.625,1.000,504.0,1898.0,5.5,14.00,24.0,88.6,30.0,38.50,52.0,103.0
199,0,0,1,0,0.597,0.006,325,600,0.625,1.000,480.0,1874.0,5.5,14.85,21.0,89.0,30.0,38.50,51.0,105.4
200,1,0,0,0,0.484,0.008,345,175,0.711,0.658,540.0,1985.0,4.0,15.05,31.0,90.5,28.0,49.25,68.0,136.0
201,0,0,0,1,0.421,0.005,350,175,0.670,1.000,616.0,2248.0,6.0,18.75,37.0,82.9,32.0,53.25,68.0,119.0


In [2]:
champ_data_df.columns = champ_data_df.columns.str.replace(" |-" , "_", regex=True)
champ_data_df.rename(columns={"Move._speed" : "Move_speed"}, inplace=True)

In [3]:
def percent_complete(df):
    print(df.notnull().mean()*100)

percent_complete(champ_data_df)

Role_of_champion_role_jungle     100.0
Role_of_champion_role_mid        100.0
Role_of_champion_role_support    100.0
Role_of_champion_role_top        100.0
Win_rate_of_champion             100.0
Pick_rate_of_champion            100.0
Move_speed                       100.0
Attack_range                     100.0
Base_AS                          100.0
AS_ratio                         100.0
Min_Health                       100.0
Max_Health                       100.0
Min_Health_regen                 100.0
Max_Health_regen                 100.0
Min_Armor                        100.0
Max_Armor                        100.0
Min_Magic_resist                 100.0
Max_Magic_resist                 100.0
Min_Attack_damage                100.0
Max_Attack_damage                100.0
dtype: float64


In [5]:
import plotly.express as px
col_lst = ["Win_rate_of_champion", "Pick_rate_of_champion", "Move_speed","Attack_range", "Base_AS", "AS_ratio", "Min_Health", "Max_Health", "Min_Health_regen","Max_Health_regen","Min_Armor","Max_Armor","Min_Magic_resist","Max_Magic_resist","Min_Attack_damage","Max_Attack_damage"]
figs = []
for col in col_lst:
    figs.append(px.histogram(champ_data_df, col, marginal="violin", title=col + ' histogram'))
for fig in figs:
    fig.show()
    

In [6]:
fig = px.imshow(champ_data_df.corr(), title="Correlation heatmap of numerical values from champion stats")
fig.update_layout(height=800)
fig.show()

In [7]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top"])
 

In [8]:
# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)
formula

'Win_rate_of_champion ~  Pick_rate_of_champion + Move_speed + Attack_range + Base_AS + AS_ratio + Min_Health + Max_Health + Min_Health_regen + Max_Health_regen + Min_Armor + Max_Armor + Min_Magic_resist + Max_Magic_resist + Min_Attack_damage + Max_Attack_damage'

In [9]:
import statsmodels.formula.api as smf

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()
#Stastically equal to 0

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.039
Model:                              OLS   Adj. R-squared:                 -0.038
Method:                   Least Squares   F-statistic:                    0.5094
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.934
Time:                          16:19:03   Log-Likelihood:                 348.71
No. Observations:                   202   AIC:                            -665.4
Df Residuals:                       186   BIC:                            -612.5
Df Model:                            15                                         
Covariance Type:              nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.8344      0.241      3.468      0.001       0.360       1.309
Pick_rate_of_champion     0.0028      0.078      0.036      0.971      -0.151       0.157
Move_speed               -0.0008      0.001     -1.384      0.168      -0.002       0.000
Attack_range          -6.274e-05   4.67e-05     -1.343      0.181      -0.000    2.94e-05
Base_AS                  -0.0638      0.105     -0.608      0.544      -0.271       0.143
AS_ratio                  0.0268      0.024      1.134      0.258      -0.020       0.074
Min_Health            -3.124e-06      0.000     -0.024      0.981      -0.000       0.000
Max_Health            -8.988e-06   2.87e-05     -0.313      0.755   -6.57e-05    4.77e-05
Min_Health_regen         -0.0006      0.003     -0.181      0.857      -0.007       0.006
Max_Health_regen         -0.0006      0.001     -0.405      0.686      -0.003       0.002
Min_Armor                -0.0011      0.001     -1.199      0.232      -0.003       0.001
Max_Armor                 0.0007      0.000      1.472      0.143      -0.000       0.002
Min_Magic_resist      -5.426e-05      0.003     -0.018      0.985      -0.006       0.006
Max_Magic_resist         -0.0001      0.001     -0.082      0.935      -0.003       0.003
Min_Attack_damage         0.0004      0.001      0.402      0.688      -0.001       0.002
Max_Attack_damage        -0.0003      0.000     -0.942      0.348      -0.001       0.000
==============================================================================
Omnibus:                       21.737   Durbin-Watson:                   1.357
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.993
Skew:                           0.460   Prob(JB):                     8.45e-12
Kurtosis:                       5.283   Cond. No.                     1.71e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.71e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.039
Model:                              OLS   Adj. R-squared:                 -0.032
Method:                   Least Squares   F-statistic:                    0.5487
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.901
Time:                          16:22:05   Log-Likelihood:                 348.71
No. Observations:                   202   AIC:                            -667.4
Df Residuals:                       187   BIC:                            -617.8
Df Model:                            14                                         
Covariance Type:              nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.8332      0.231      3.608      0.000       0.378       1.289
Pick_rate_of_champion     0.0028      0.078      0.036      0.971      -0.151       0.156
Move_speed               -0.0008      0.001     -1.387      0.167      -0.002       0.000
Attack_range          -6.293e-05   4.55e-05     -1.384      0.168      -0.000    2.68e-05
Base_AS                  -0.0637      0.104     -0.610      0.543      -0.270       0.142
AS_ratio                  0.0268      0.024      1.138      0.257      -0.020       0.073
Min_Health            -3.354e-06      0.000     -0.025      0.980      -0.000       0.000
Max_Health            -8.944e-06   2.86e-05     -0.313      0.755   -6.53e-05    4.74e-05
Min_Health_regen         -0.0006      0.003     -0.182      0.856      -0.007       0.005
Max_Health_regen         -0.0006      0.001     -0.406      0.685      -0.003       0.002
Min_Armor                -0.0011      0.001     -1.212      0.227      -0.003       0.001
Max_Armor                 0.0007      0.000      1.489      0.138      -0.000       0.002
Max_Magic_resist         -0.0001      0.001     -0.114      0.909      -0.002       0.002
Min_Attack_damage         0.0004      0.001      0.403      0.687      -0.001       0.002
Max_Attack_damage        -0.0003      0.000     -0.945      0.346      -0.001       0.000
==============================================================================
Omnibus:                       21.747   Durbin-Watson:                   1.357
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               51.010
Skew:                           0.460   Prob(JB):                     8.38e-12
Kurtosis:                       5.283   Cond. No.                     1.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.039
Model:                              OLS   Adj. R-squared:                 -0.027
Method:                   Least Squares   F-statistic:                    0.5940
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.857
Time:                          16:22:55   Log-Likelihood:                 348.71
No. Observations:                   202   AIC:                            -669.4
Df Residuals:                       188   BIC:                            -623.1
Df Model:                            13                                         
Covariance Type:              nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.8318      0.224      3.714      0.000       0.390       1.274
Pick_rate_of_champion     0.0027      0.078      0.035      0.972      -0.150       0.156
Move_speed               -0.0008      0.001     -1.391      0.166      -0.002       0.000
Attack_range          -6.267e-05   4.42e-05     -1.418      0.158      -0.000    2.45e-05
Base_AS                  -0.0636      0.104     -0.611      0.542      -0.269       0.142
AS_ratio                  0.0268      0.023      1.141      0.255      -0.020       0.073
Max_Health            -9.324e-06   2.43e-05     -0.384      0.701   -5.72e-05    3.85e-05
Min_Health_regen         -0.0006      0.003     -0.183      0.855      -0.007       0.005
Max_Health_regen         -0.0006      0.001     -0.406      0.685      -0.003       0.002
Min_Armor                -0.0012      0.001     -1.293      0.197      -0.003       0.001
Max_Armor                 0.0007      0.000      1.502      0.135      -0.000       0.002
Max_Magic_resist         -0.0001      0.001     -0.112      0.911      -0.002       0.002
Min_Attack_damage         0.0004      0.001      0.404      0.686      -0.001       0.002
Max_Attack_damage        -0.0003      0.000     -0.947      0.345      -0.001       0.000
==============================================================================
Omnibus:                       21.707   Durbin-Watson:                   1.356
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.938
Skew:                           0.459   Prob(JB):                     8.69e-12
Kurtosis:                       5.282   Cond. No.                     1.56e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.56e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.039
Model:                              OLS   Adj. R-squared:                 -0.022
Method:                   Least Squares   F-statistic:                    0.6468
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.800
Time:                          16:23:35   Log-Likelihood:                 348.71
No. Observations:                   202   AIC:                            -671.4
Df Residuals:                       189   BIC:                            -628.4
Df Model:                            12                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8328      0.222      3.758      0.000       0.396       1.270
Move_speed           -0.0008      0.001     -1.407      0.161      -0.002       0.000
Attack_range      -6.261e-05   4.41e-05     -1.421      0.157      -0.000    2.43e-05
Base_AS              -0.0639      0.104     -0.616      0.538      -0.268       0.141
AS_ratio              0.0268      0.023      1.144      0.254      -0.019       0.073
Max_Health         -9.32e-06   2.42e-05     -0.385      0.701   -5.71e-05    3.84e-05
Min_Health_regen     -0.0006      0.003     -0.188      0.851      -0.007       0.005
Max_Health_regen     -0.0005      0.001     -0.406      0.685      -0.003       0.002
Min_Armor            -0.0011      0.001     -1.308      0.193      -0.003       0.001
Max_Armor             0.0007      0.000      1.521      0.130      -0.000       0.002
Max_Magic_resist     -0.0001      0.001     -0.111      0.912      -0.002       0.002
Min_Attack_damage     0.0004      0.001      0.411      0.682      -0.001       0.002
Max_Attack_damage    -0.0003      0.000     -0.955      0.341      -0.001       0.000
==============================================================================
Omnibus:                       21.561   Durbin-Watson:                   1.354
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.601
Skew:                           0.455   Prob(JB):                     1.03e-11
Kurtosis:                       5.277   Cond. No.                     1.54e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.54e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.039
Model:                              OLS   Adj. R-squared:                 -0.016
Method:                   Least Squares   F-statistic:                    0.7082
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.730
Time:                          16:25:37   Log-Likelihood:                 348.71
No. Observations:                   202   AIC:                            -673.4
Df Residuals:                       190   BIC:                            -633.7
Df Model:                            11                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8305      0.220      3.773      0.000       0.396       1.265
Move_speed           -0.0008      0.001     -1.428      0.155      -0.002       0.000
Attack_range      -5.955e-05   3.42e-05     -1.740      0.084      -0.000    7.98e-06
Base_AS              -0.0624      0.103     -0.609      0.543      -0.265       0.140
AS_ratio              0.0269      0.023      1.155      0.250      -0.019       0.073
Max_Health        -9.789e-06   2.38e-05     -0.412      0.681   -5.67e-05    3.71e-05
Min_Health_regen     -0.0006      0.003     -0.195      0.846      -0.007       0.005
Max_Health_regen     -0.0006      0.001     -0.432      0.667      -0.003       0.002
Min_Armor            -0.0011      0.001     -1.309      0.192      -0.003       0.001
Max_Armor             0.0007      0.000      1.545      0.124      -0.000       0.002
Min_Attack_damage     0.0004      0.001      0.397      0.692      -0.001       0.002
Max_Attack_damage    -0.0003      0.000     -0.956      0.340      -0.001       0.000
==============================================================================
Omnibus:                       21.526   Durbin-Watson:                   1.354
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.429
Skew:                           0.455   Prob(JB):                     1.12e-11
Kurtosis:                       5.272   Cond. No.                     1.54e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.54e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.039
Model:                              OLS   Adj. R-squared:                 -0.011
Method:                   Least Squares   F-statistic:                    0.7791
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.649
Time:                          16:27:46   Log-Likelihood:                 348.69
No. Observations:                   202   AIC:                            -675.4
Df Residuals:                       191   BIC:                            -639.0
Df Model:                            10                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8298      0.220      3.780      0.000       0.397       1.263
Move_speed           -0.0008      0.001     -1.457      0.147      -0.002       0.000
Attack_range      -5.733e-05   3.22e-05     -1.781      0.077      -0.000    6.17e-06
Base_AS              -0.0587      0.100     -0.584      0.560      -0.257       0.139
AS_ratio              0.0268      0.023      1.153      0.250      -0.019       0.073
Max_Health        -1.006e-05   2.37e-05     -0.425      0.671   -5.67e-05    3.66e-05
Max_Health_regen     -0.0007      0.001     -0.686      0.493      -0.003       0.001
Min_Armor            -0.0011      0.001     -1.301      0.195      -0.003       0.001
Max_Armor             0.0007      0.000      1.538      0.126      -0.000       0.002
Min_Attack_damage     0.0004      0.001      0.414      0.680      -0.001       0.002
Max_Attack_damage    -0.0003      0.000     -0.955      0.341      -0.001       0.000
==============================================================================
Omnibus:                       20.939   Durbin-Watson:                   1.355
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.890
Skew:                           0.441   Prob(JB):                     2.42e-11
Kurtosis:                       5.243   Cond. No.                     1.53e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.038
Model:                              OLS   Adj. R-squared:                 -0.007
Method:                   Least Squares   F-statistic:                    0.8504
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.571
Time:                          16:28:24   Log-Likelihood:                 348.60
No. Observations:                   202   AIC:                            -677.2
Df Residuals:                       192   BIC:                            -644.1
Df Model:                             9                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8298      0.219      3.788      0.000       0.398       1.262
Move_speed           -0.0008      0.001     -1.414      0.159      -0.002       0.000
Attack_range      -5.888e-05   3.19e-05     -1.846      0.066      -0.000    4.05e-06
Base_AS              -0.0562      0.100     -0.562      0.575      -0.253       0.141
AS_ratio              0.0270      0.023      1.163      0.246      -0.019       0.073
Max_Health        -9.476e-06   2.36e-05     -0.402      0.688    -5.6e-05     3.7e-05
Max_Health_regen     -0.0007      0.001     -0.693      0.489      -0.003       0.001
Min_Armor            -0.0010      0.001     -1.239      0.217      -0.003       0.001
Max_Armor             0.0006      0.000      1.490      0.138      -0.000       0.001
Max_Attack_damage    -0.0002      0.000     -0.868      0.386      -0.001       0.000
==============================================================================
Omnibus:                       21.100   Durbin-Watson:                   1.362
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.238
Skew:                           0.437   Prob(JB):                     1.23e-11
Kurtosis:                       5.281   Cond. No.                     1.53e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage", "Max_Health"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.038
Model:                              OLS   Adj. R-squared:                 -0.002
Method:                   Least Squares   F-statistic:                    0.9406
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.484
Time:                          16:29:50   Log-Likelihood:                 348.51
No. Observations:                   202   AIC:                            -679.0
Df Residuals:                       193   BIC:                            -649.2
Df Model:                             8                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8082      0.212      3.814      0.000       0.390       1.226
Move_speed           -0.0008      0.001     -1.401      0.163      -0.002       0.000
Attack_range      -5.589e-05    3.1e-05     -1.805      0.073      -0.000    5.17e-06
Base_AS              -0.0560      0.100     -0.561      0.576      -0.253       0.141
AS_ratio              0.0259      0.023      1.127      0.261      -0.019       0.071
Max_Health_regen     -0.0007      0.001     -0.702      0.483      -0.003       0.001
Min_Armor            -0.0010      0.001     -1.189      0.236      -0.003       0.001
Max_Armor             0.0006      0.000      1.469      0.144      -0.000       0.001
Max_Attack_damage    -0.0003      0.000     -0.938      0.350      -0.001       0.000
==============================================================================
Omnibus:                       20.857   Durbin-Watson:                   1.362
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.642
Skew:                           0.431   Prob(JB):                     1.66e-11
Kurtosis:                       5.271   Cond. No.                     3.49e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.49e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage", "Max_Health", "Base_AS"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.036
Model:                              OLS   Adj. R-squared:                  0.001
Method:                   Least Squares   F-statistic:                     1.034
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.409
Time:                          16:30:49   Log-Likelihood:                 348.35
No. Observations:                   202   AIC:                            -680.7
Df Residuals:                       194   BIC:                            -654.2
Df Model:                             7                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.7625      0.195      3.905      0.000       0.377       1.148
Move_speed           -0.0008      0.001     -1.383      0.168      -0.002       0.000
Attack_range       -5.29e-05   3.04e-05     -1.738      0.084      -0.000    7.14e-06
AS_ratio              0.0307      0.021      1.435      0.153      -0.011       0.073
Max_Health_regen     -0.0007      0.001     -0.632      0.528      -0.003       0.001
Min_Armor            -0.0010      0.001     -1.202      0.231      -0.003       0.001
Max_Armor             0.0006      0.000      1.485      0.139      -0.000       0.001
Max_Attack_damage    -0.0003      0.000     -0.996      0.320      -0.001       0.000
==============================================================================
Omnibus:                       21.065   Durbin-Watson:                   1.353
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.111
Skew:                           0.436   Prob(JB):                     1.31e-11
Kurtosis:                       5.279   Cond. No.                     3.16e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.16e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage", "Max_Health", "Base_AS", "Max_Health_regen"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.034
Model:                              OLS   Adj. R-squared:                  0.004
Method:                   Least Squares   F-statistic:                     1.143
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.339
Time:                          16:32:27   Log-Likelihood:                 348.14
No. Observations:                   202   AIC:                            -682.3
Df Residuals:                       195   BIC:                            -659.1
Df Model:                             6                                         
Covariance Type:              nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.7662      0.195      3.932      0.000       0.382       1.151
Move_speed           -0.0008      0.001     -1.476      0.142      -0.002       0.000
Attack_range      -4.748e-05   2.92e-05     -1.628      0.105      -0.000       1e-05
AS_ratio              0.0299      0.021      1.403      0.162      -0.012       0.072
Min_Armor            -0.0009      0.001     -1.173      0.242      -0.003       0.001
Max_Armor             0.0006      0.000      1.440      0.152      -0.000       0.001
Max_Attack_damage    -0.0003      0.000     -1.020      0.309      -0.001       0.000
==============================================================================
Omnibus:                       21.112   Durbin-Watson:                   1.357
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.565
Skew:                           0.443   Prob(JB):                     1.73e-11
Kurtosis:                       5.259   Cond. No.                     3.15e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.15e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage", "Max_Health", "Base_AS", "Max_Health_regen", "Max_Attack_damage"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.029
Model:                              OLS   Adj. R-squared:                  0.004
Method:                   Least Squares   F-statistic:                     1.163
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.329
Time:                          16:32:56   Log-Likelihood:                 347.60
No. Observations:                   202   AIC:                            -683.2
Df Residuals:                       196   BIC:                            -663.4
Df Model:                             5                                         
Covariance Type:              nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.7564      0.195      3.886      0.000       0.373       1.140
Move_speed      -0.0009      0.001     -1.565      0.119      -0.002       0.000
Attack_range -3.737e-05   2.74e-05     -1.362      0.175   -9.15e-05    1.67e-05
AS_ratio         0.0246      0.021      1.191      0.235      -0.016       0.065
Min_Armor       -0.0010      0.001     -1.218      0.225      -0.003       0.001
Max_Armor        0.0006      0.000      1.328      0.186      -0.000       0.001
==============================================================================
Omnibus:                       21.473   Durbin-Watson:                   1.355
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.411
Skew:                           0.453   Prob(JB):                     1.13e-11
Kurtosis:                       5.274   Cond. No.                     3.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage", "Max_Health", "Base_AS", "Max_Health_regen", "Max_Attack_damage", "AS_ratio"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.022
Model:                              OLS   Adj. R-squared:                  0.002
Method:                   Least Squares   F-statistic:                     1.097
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.359
Time:                          16:33:25   Log-Likelihood:                 346.87
No. Observations:                   202   AIC:                            -683.7
Df Residuals:                       197   BIC:                            -667.2
Df Model:                             4                                         
Covariance Type:              nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.7521      0.195      3.860      0.000       0.368       1.136
Move_speed      -0.0008      0.001     -1.416      0.158      -0.002       0.000
Attack_range -3.516e-05   2.74e-05     -1.284      0.201   -8.92e-05    1.89e-05
Min_Armor       -0.0010      0.001     -1.201      0.231      -0.003       0.001
Max_Armor        0.0005      0.000      1.238      0.217      -0.000       0.001
==============================================================================
Omnibus:                       23.017   Durbin-Watson:                   1.343
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.381
Skew:                           0.483   Prob(JB):                     9.42e-13
Kurtosis:                       5.376   Cond. No.                     3.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage", "Max_Health", "Base_AS", "Max_Health_regen", "Max_Attack_damage", "AS_ratio", "Min_Armor"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.015
Model:                              OLS   Adj. R-squared:                 -0.000
Method:                   Least Squares   F-statistic:                    0.9798
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.403
Time:                          16:34:34   Log-Likelihood:                 346.14
No. Observations:                   202   AIC:                            -684.3
Df Residuals:                       198   BIC:                            -671.0
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.7395      0.195      3.797      0.000       0.355       1.124
Move_speed      -0.0008      0.001     -1.397      0.164      -0.002       0.000
Attack_range -1.818e-05   2.35e-05     -0.774      0.440   -6.45e-05    2.81e-05
Max_Armor        0.0002      0.000      0.690      0.491      -0.000       0.001
==============================================================================
Omnibus:                       22.656   Durbin-Watson:                   1.384
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               54.117
Skew:                           0.476   Prob(JB):                     1.77e-12
Kurtosis:                       5.350   Cond. No.                     3.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage", "Max_Health", "Base_AS", "Max_Health_regen", "Max_Attack_damage", "AS_ratio", "Min_Armor", "Max_Armor"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.012
Model:                              OLS   Adj. R-squared:                  0.002
Method:                   Least Squares   F-statistic:                     1.235
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.293
Time:                          16:34:55   Log-Likelihood:                 345.89
No. Observations:                   202   AIC:                            -685.8
Df Residuals:                       199   BIC:                            -675.9
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.7820      0.185      4.239      0.000       0.418       1.146
Move_speed      -0.0008      0.001     -1.517      0.131      -0.002       0.000
Attack_range -2.603e-05   2.05e-05     -1.269      0.206   -6.65e-05    1.44e-05
==============================================================================
Omnibus:                       22.260   Durbin-Watson:                   1.371
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.171
Skew:                           0.474   Prob(JB):                     4.69e-12
Kurtosis:                       5.302   Cond. No.                     2.86e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.86e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
win_drop = champ_data_df.columns.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top", "Min_Magic_resist", "Min_Health", "Pick_rate_of_champion", "Max_Magic_resist", "Min_Health_regen", "Min_Attack_damage", "Max_Health", "Base_AS", "Max_Health_regen", "Max_Attack_damage", "AS_ratio", "Min_Armor", "Max_Armor", "Attack_range"])

# y_target ~ xcol1 + xcol2 + xcol3 ...
formula = "Win_rate_of_champion ~  " + " + ".join(win_drop)

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.004
Model:                              OLS   Adj. R-squared:                 -0.001
Method:                   Least Squares   F-statistic:                    0.8579
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.355
Time:                          16:36:06   Log-Likelihood:                 345.08
No. Observations:                   202   AIC:                            -686.2
Df Residuals:                       200   BIC:                            -679.5
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6308      0.141      4.473      0.000       0.353       0.909
Move_speed    -0.0004      0.000     -0.926      0.355      -0.001       0.000
==============================================================================
Omnibus:                       23.554   Durbin-Watson:                   1.353
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               58.739
Skew:                           0.483   Prob(JB):                     1.76e-13
Kurtosis:                       5.459   Cond. No.                     1.53e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [45]:
# Look at catagorical roles
# Split data into train, test, validate
# Go back and add more variables?

In [27]:
formula = "Win_rate_of_champion ~  " + " + ".join(champ_data_df.columns.drop(["Win_rate_of_champion"]))

baseline_model = smf.ols(formula, champ_data_df).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     Win_rate_of_champion   R-squared:                       0.043
Model:                              OLS   Adj. R-squared:                 -0.057
Method:                   Least Squares   F-statistic:                    0.4334
Date:                  Tue, 06 Apr 2021   Prob (F-statistic):              0.982
Time:                          16:38:05   Log-Likelihood:                 349.12
No. Observations:                   202   AIC:                            -658.2
Df Residuals:                       182   BIC:                            -592.1
Df Model:                            19                                         
Covariance Type:              nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         0.8727      0.250      3.492      0.001       0.380       1.366
Role_of_champion_role_jungle      0.0030      0.016      0.185      0.853      -0.029       0.035
Role_of_champion_role_mid         0.0042      0.016      0.263      0.793      -0.027       0.035
Role_of_champion_role_support    -0.0048      0.016     -0.298      0.766      -0.037       0.027
Role_of_champion_role_top         0.0008      0.016      0.053      0.958      -0.031       0.032
Pick_rate_of_champion             0.0068      0.082      0.083      0.934      -0.155       0.168
Move_speed                       -0.0009      0.001     -1.434      0.153      -0.002       0.000
Attack_range                  -6.215e-05   4.81e-05     -1.292      0.198      -0.000    3.28e-05
Base_AS                          -0.0724      0.107     -0.679      0.498      -0.283       0.138
AS_ratio                          0.0272      0.024      1.135      0.258      -0.020       0.075
Min_Health                    -2.675e-05      0.000     -0.190      0.849      -0.000       0.000
Max_Health                    -7.531e-06   2.96e-05     -0.254      0.800    -6.6e-05    5.09e-05
Min_Health_regen                 -0.0006      0.003     -0.172      0.864      -0.007       0.006
Max_Health_regen                 -0.0007      0.001     -0.521      0.603      -0.004       0.002
Min_Armor                        -0.0009      0.001     -0.856      0.393      -0.003       0.001
Max_Armor                         0.0007      0.000      1.443      0.151      -0.000       0.002
Min_Magic_resist              -5.862e-05      0.003     -0.020      0.984      -0.006       0.006
Max_Magic_resist                 -0.0001      0.001     -0.096      0.924      -0.003       0.003
Min_Attack_damage                 0.0003      0.001      0.302      0.763      -0.002       0.002
Max_Attack_damage                -0.0003      0.000     -0.947      0.345      -0.001       0.000
==============================================================================
Omnibus:                       20.421   Durbin-Watson:                   1.359
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               45.194
Skew:                           0.448   Prob(JB):                     1.54e-10
Kurtosis:                       5.137   Cond. No.                     1.76e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [48]:
from sklearn.model_selection import train_test_split

X = champ_data_df.drop(["Win_rate_of_champion"], axis=1).values
y = champ_data_df["Win_rate_of_champion"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.5)

print("Number of samples in:")
print(f"    Training: {len(y_train)}")
print(f"    Validation: {len(y_validation)}")
print(f"    Testing: {len(y_test)}")

Number of samples in:
    Training: 141
    Validation: 31
    Testing: 30


In [53]:
# ML algorithms you will later know, don't panic
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

np.random.seed(2)

models = [LinearRegression(normalize = True),SVR(),DecisionTreeRegressor(splitter='random')]
# SVR.fit()
for model in models:
    print(model)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    valid_pred = model.predict(X_validation)
    train_loss = mean_squared_error(y_train, train_pred)
    test_loss = mean_squared_error(y_test, test_pred)
    validation_loss = mean_squared_error(y_validation, valid_pred)

    print(
        f"{model.__class__.__name__}: "
        f"Train Loss: {train_loss} | Validation Loss: {validation_loss} | "
        f"Test Loss: {test_loss}"
    )

LinearRegression(normalize=True)
LinearRegression: Train Loss: 0.0018890011665305176 | Validation Loss: 0.0018927097398923134 | Test Loss: 0.0021404615366632284
SVR()
SVR: Train Loss: 0.0022404353846143155 | Validation Loss: 0.002073972628397357 | Test Loss: 0.0022642934974430773
DecisionTreeRegressor(splitter='random')
DecisionTreeRegressor: Train Loss: 0.0 | Validation Loss: 0.0068021290322580665 | Test Loss: 0.00518866666666667


In [62]:
from sklearn.model_selection import train_test_split

X = champ_data_df.drop(["Win_rate_of_champion", "Role_of_champion_role_jungle", "Role_of_champion_role_mid", "Role_of_champion_role_support", "Role_of_champion_role_top"], axis=1).values
y = champ_data_df["Win_rate_of_champion"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.5)

print("Number of samples in:")
print(f"    Training: {len(y_train)}")
print(f"    Validation: {len(y_validation)}")
print(f"    Testing: {len(y_test)}")

Number of samples in:
    Training: 141
    Validation: 31
    Testing: 30


In [63]:
# ML algorithms you will later know, don't panic
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

np.random.seed(2)

models = [LinearRegression(normalize = True),SVR(),DecisionTreeRegressor(splitter='random')]
# SVR.fit()
for model in models:
    print(model)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    valid_pred = model.predict(X_validation)
    train_loss = mean_squared_error(y_train, train_pred)
    test_loss = mean_squared_error(y_test, test_pred)
    validation_loss = mean_squared_error(y_validation, valid_pred)

    print(
        f"{model.__class__.__name__}: "
        f"Train Loss: {train_loss} | Validation Loss: {validation_loss} | "
        f"Test Loss: {test_loss}"
    )

LinearRegression(normalize=True)
LinearRegression: Train Loss: 0.0017352701452230754 | Validation Loss: 0.0023632288407662927 | Test Loss: 0.002270054804166567
SVR()
SVR: Train Loss: 0.002116058325051524 | Validation Loss: 0.002497387426460987 | Test Loss: 0.0021550067560276985
DecisionTreeRegressor(splitter='random')
DecisionTreeRegressor: Train Loss: 2.2163120567375924e-06 | Validation Loss: 0.0030970645161290332 | Test Loss: 0.004160266666666667


In [ ]:
# Remember to say about it could be the player skill that is the highest factor
# Also that you dropped Jhin
# Now with new data repeat the process